## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ferrol,ES,2021-11-08 02:47:23,43.4832,-8.2369,55.72,81,95,4.23,overcast clouds
1,1,Pisco,PE,2021-11-08 02:45:49,-13.7000,-76.2167,65.01,82,0,5.75,clear sky
2,2,Tasiilaq,GL,2021-11-08 02:47:24,65.6145,-37.6368,26.76,75,100,5.97,overcast clouds
3,3,Batagay,RU,2021-11-08 02:47:24,67.6333,134.6333,-13.29,85,34,5.77,scattered clouds
4,4,Castro,BR,2021-11-08 02:47:18,-24.7911,-50.0119,58.82,96,88,6.89,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
5,5,Takoradi,GH,2021-11-08 02:47:24,4.8845,-1.7554,78.12,88,77,6.96,broken clouds
6,6,Soure,BR,2021-11-08 02:47:25,-0.7167,-48.5233,80.47,78,59,7.67,broken clouds
9,9,Hithadhoo,MV,2021-11-08 02:47:26,-0.6000,73.0833,82.24,72,98,13.00,overcast clouds
10,10,Mogadishu,SO,2021-11-08 02:47:26,2.0371,45.3438,78.60,78,100,8.99,overcast clouds
12,12,Souillac,MU,2021-11-08 02:47:27,-20.5167,57.5167,77.36,83,40,8.05,scattered clouds
17,17,Port Hedland,AU,2021-11-08 02:47:28,-20.3167,118.5667,82.45,54,30,19.57,scattered clouds
28,28,Kinablangan,PH,2021-11-08 02:47:32,7.6947,126.5503,85.42,71,7,4.65,clear sky
33,33,Kapaa,US,2021-11-08 02:47:33,22.0752,-159.3190,84.18,71,40,3.00,scattered clouds
37,37,Atuona,PF,2021-11-08 02:47:29,-9.8000,-139.0333,78.39,75,9,16.71,clear sky
46,46,Vaini,TO,2021-11-08 02:48:37,-21.2000,-175.2000,78.96,94,75,5.75,moderate rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        198
City           198
Country        198
Date           198
Lat            198
Lng            198
Max Temp       198
Humidity       198
Cloudiness     198
Wind Speed     198
Description    198
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
5,Takoradi,GH,78.12,broken clouds,4.8845,-1.7554,
6,Soure,BR,80.47,broken clouds,-0.7167,-48.5233,
9,Hithadhoo,MV,82.24,overcast clouds,-0.6000,73.0833,
10,Mogadishu,SO,78.60,overcast clouds,2.0371,45.3438,
12,Souillac,MU,77.36,scattered clouds,-20.5167,57.5167,
17,Port Hedland,AU,82.45,scattered clouds,-20.3167,118.5667,
28,Kinablangan,PH,85.42,clear sky,7.6947,126.5503,
33,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,
37,Atuona,PF,78.39,clear sky,-9.8000,-139.0333,
46,Vaini,TO,78.96,moderate rain,-21.2000,-175.2000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [67]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [69]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [73]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Conditions</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [75]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))